In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel
from datasets import load_dataset
import numpy as np
import pandas as pd

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
df = pd.read_csv("/content/spam.csv")
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df.Category = df.Category.map({'ham':0, 'spam':1})
#mappinng ham and spam to 0,1
df.head()

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df_spam = df[df.Category == 1]
#df[] syntax is used to select rows where the condition Category == 1 is True
df_spam

,Category,Message
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
5,1,FreeMsg Hey there darling it's been 3 week's n...
8,1,WINNER!! As a valued network customer you have...
9,1,Had your mobile 11 months or more? U R entitle...
11,1,"SIX chances to win CASH! From 100 to 20,000 po..."
...,...,...
5537,1,Want explicit SEX in 30 secs? Ring 02073162414...
5540,1,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,1,Had your contract mobile 11 Mnths? Latest Moto...
5566,1,REMINDER FROM O2: To get 2.50 pounds free call...


In [ ]:
df_ham = df[df.Category==0].sample(1000)
df_ham

,Category,Message
47,0,"Fair enough, anything going on?"
5549,0,"You know, wot people wear. T shirts, jumpers, ..."
4878,0,The world's most happiest frnds never have the...
4397,0,Dear :-/ why you mood off. I cant drive so i b...
1131,0,Howz pain?hope u r fine..
...,...,...
1508,0,"Wen ur lovable bcums angry wid u, dnt take it ..."
3440,0,"awesome, how do I deal with the gate? Charles ..."
3262,0,Yup i'm still having coffee wif my frens... My...
3105,0,That's y i said it's bad dat all e gals know u...


In [ ]:
df_small = pd.concat([df_spam, df_ham])
df_small

,Category,Message
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
5,1,FreeMsg Hey there darling it's been 3 week's n...
8,1,WINNER!! As a valued network customer you have...
9,1,Had your mobile 11 months or more? U R entitle...
11,1,"SIX chances to win CASH! From 100 to 20,000 po..."
...,...,...
1508,0,"Wen ur lovable bcums angry wid u, dnt take it ..."
3440,0,"awesome, how do I deal with the gate? Charles ..."
3262,0,Yup i'm still having coffee wif my frens... My...
3105,0,That's y i said it's bad dat all e gals know u...


In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_small.Message,
                                                    df_small.Category,
                                                    test_size=0.2,
                                                    random_state=5)

#random_state=5 is making sure that training and test sets stay the same every time the code is runned.

In [34]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#creatinng contexual embedding
def tokenize_function(text, labels):
  encodings = tokenizer(text,
                        padding="max_length",
                        max_length=256,
                        truncation=True,
                        return_tensors='pt'
                        )
  return encodings['input_ids'], encodings['attention_mask'], torch.tensor(labels, dtype=torch.float)
  #input_ids and attention_mask are the two core inputs for BERT-like models
  #Returning just encodings is possible, but then extra handling is needed later.
  #torch.tensor(labels, dtype=torch.float()) are my tarrget vallues to predict

In [25]:
X_train.values.tolist()
y_train.values.tolist()

'''tokenizer and your tokenize_function expect standard Python types
(list, str, int, etc.), not Pandas-specific objects like df
That's why X_train and y_train are getting converted into list'''

"tokenizer and your tokenize_function expect standard Python types \n(list, str, int, etc.), not Pandas-specific objects like df\nThat's why X_train and y_train are getting converted into list"

In [36]:
train_input_ids, train_attention_mask, train_lebels = tokenize_function(X_train.values.tolist(), y_train.values.tolist())
test_input_ids, test_attention_mask, test_lebels = tokenize_function(X_test.values.tolist(), y_test.values.tolist())

In [38]:
train_dataset = torch.utils.data.TensorDataset(train_input_ids, train_attention_mask, train_lebels)
test_dataset = torch.utils.data.TensorDataset(test_input_ids, test_attention_mask, test_lebels)
#tensors are taken and zips them together into a dataset
#Each index in the dataset returns a tuple containing the tensors at that position (input and labels)

In [39]:
batch_size=48

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#Shuffling (if shuffle=True): 1. Randomizes the order of data each epoch, 2. Prevents the model from memorizing the order of samples

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#DataLoader splits the dataset into batches of size batch_size=48
#Basically DataLoader pura data shonngkha k 48 ta data'r bin'e bhag kortese

In [40]:
bert = BertModel.from_pretrained("bert-base-uncased")
bert.config.hidden_size
#size of the hf model

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

768

In [50]:
class SentimentClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.bert = BertModel.from_pretrained("bert-base-uncased")

    for param in self.bert.parameters():
      param.requires_grad = False
      #We are freeziing all the layers as BERT layers are already highly trained
      #We don't need to update gradient
      self.classifier=nn.Sequential(
          nn.Linear(bert.config.hidden_size, 256),

          #we will have to provide the bert.config.hidden_size for assigning the inputs equivalent to the model size
          #IMPORTANT::: nn.Linear = just one building block (a dense layer)
          #inside a neural network. It can perform linear regression
          #if it’s the final layer with no activation,
          #but usually it’s followed by nonlinear activations
          #(like ReLU, Sigmoid, or Softmax),
          #making the whole network non-linear.'''

          nn.ReLU(),
          nn.Dropout(0.3),
          #to avoid overfitting and model optimization
          nn.Linear(256, 1),
          nn.Sigmoid()
          #using sigmlid bc our output will from 0-1 (spam or ham)
      )

  def forward(self, input_ids, attention_mask):
    bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
    sentence_embedding =  bert_output.last_hidden_state[:,0,:]
    return self.classifier(sentence_embedding)

In [51]:
model = SentimentClassifier()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()
#BCE bc binary classification

model=model.to(device)
criterion=criterion.to(device)

# **Training the Model**

In [52]:
epochs=5

for epoch in range(epochs):
  model.train()
  total_training_loss=0

  for batch, (input_ids, attention_mask, labels) in enumerate(train_loader):
    input_ids=input_ids.to(device)
    attention_mask=attention_mask.to(device)
    labels=labels.to(device, dtype=torch.float)

    optimizer.zero_grad()
    outputs=model(input_ids, attention_mask).squeeze()

    loss=criterion(outputs, labels)
    loss.backward()

    optimizer.step()


    print(f"Batch: {batch}, Epoch:{epoch}, Loss:{loss.item(): 0.2f}")
    total_training_loss += loss.item()

  avg_training_loss=total_training_loss/len(train_loader)
  print(f'Epoch {epoch+1}/{epochs}, Training Loss: {avg_training_loss}: 0.4f')

Batch: 0, Epoch:0, Loss: 0.69
Batch: 1, Epoch:0, Loss: 0.62
Batch: 2, Epoch:0, Loss: 0.56
Batch: 3, Epoch:0, Loss: 0.52
Batch: 4, Epoch:0, Loss: 0.54
Batch: 5, Epoch:0, Loss: 0.39
Batch: 6, Epoch:0, Loss: 0.35
Batch: 7, Epoch:0, Loss: 0.36
Batch: 8, Epoch:0, Loss: 0.40
Batch: 9, Epoch:0, Loss: 0.27
Batch: 10, Epoch:0, Loss: 0.30
Batch: 11, Epoch:0, Loss: 0.22
Batch: 12, Epoch:0, Loss: 0.28
Batch: 13, Epoch:0, Loss: 0.15
Batch: 14, Epoch:0, Loss: 0.17
Batch: 15, Epoch:0, Loss: 0.13
Batch: 16, Epoch:0, Loss: 0.14
Batch: 17, Epoch:0, Loss: 0.16
Batch: 18, Epoch:0, Loss: 0.13
Batch: 19, Epoch:0, Loss: 0.17
Batch: 20, Epoch:0, Loss: 0.10
Batch: 21, Epoch:0, Loss: 0.03
Batch: 22, Epoch:0, Loss: 0.13
Batch: 23, Epoch:0, Loss: 0.16
Batch: 24, Epoch:0, Loss: 0.06
Batch: 25, Epoch:0, Loss: 0.06
Batch: 26, Epoch:0, Loss: 0.06
Batch: 27, Epoch:0, Loss: 0.14
Batch: 28, Epoch:0, Loss: 0.05
Batch: 29, Epoch:0, Loss: 0.08
Epoch 1/5, Training Loss: 0.24692741421361764
Batch: 0, Epoch:1, Loss: 0.07
Batc

# **Evalluation of the model**

In [53]:
model.eval()
total_val_loss = 0
correct_predictions = 0

with torch.no_grad():
    for input_ids, attention_mask, labels in test_loader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device, dtype=torch.float)

        outputs = model(input_ids, attention_mask).squeeze()
        loss = criterion(outputs, labels.view_as(outputs))
        total_val_loss += loss.item()

        preds = (outputs > 0.5).float()
        correct_predictions += torch.sum(preds == labels)

avg_val_loss = total_val_loss / len(test_loader)
val_accuracy = correct_predictions.double() / len(test_dataset)
print(f'Validation Loss: {avg_val_loss}, Validation Accuracy: {val_accuracy:.4f}')

Validation Loss: 0.07245988107752055, Validation Accuracy: 0.9686


# **Inference**

In [54]:
def predict(model, text, max_length=128):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    # Tokenize input text
    encoding = tokenizer(
        text,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    model = model.to(device)
    model.eval()
    with torch.no_grad():
        output = model(input_ids, attention_mask).squeeze()
        prediction = (output > 0.5).float().item()
        return 'spam' if prediction == 1 else 'ham'

In [55]:
predict(model, "This is your last chance to WIN cash. Click this link")

'spam'

In [56]:
predict(model, "Dear Dhaval, I hope to see you on Monday")

'ham'

In [57]:
predict(model, "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005.")

'spam'